# 전국 공영 장례 조례집 전처리 및 vectorDB 적재

In [3]:
from glob import glob

ordinances_path_list = glob(pathname="*.txt", root_dir='../data/raw/ordinance')
ordinances_path_list

['가평군 공영장례 지원에 관한 조례.txt',
 '강릉시 공영장례 지원에 관한 조례.txt',
 '강화군 공영장례 지원 조례.txt',
 '거창군 공영장례 지원에 관한 조례.txt',
 '경산시 공영장례 지원에 관한 조례.txt',
 '경주시 공영장례에 관한 조례.txt',
 '계룡시 공영장례 지원에 관한 조례.txt',
 '고령군 공영장례 지원에 관한 조례.txt',
 '고성군 공영장례 지원 조례 (1).txt',
 '고성군 공영장례 지원 조례.txt',
 '고양시 공영장례 지원에 관한 조례.txt',
 '고흥군 공영장례에 관한 조례.txt',
 '공주시 공영장례 지원에 관한 조례.txt',
 '과천시 공영장례 지원 조례.txt',
 '광명시 공영장례 운영에 관한 조례.txt',
 '광주광역시 광산구 공영장례 지원 조례.txt',
 '광주광역시 남구 공영장례 지원에 관한 조례.txt',
 '광주광역시 동구 공영장례 지원에 관한 조례.txt',
 '광주광역시 북구 공영장례지원에 관한 조례.txt',
 '광주광역시 서구 무연고 사망자 등 공영장례지원에 관한 조례.txt',
 '광주시 공영장례 지원에 관한 조례.txt',
 '괴산군 공영장례 지원에 관한 조례.txt',
 '구리시 공영장례 지원에 관한 조례.txt',
 '구미시 공영장례 지원에 관한 조례.txt',
 '군산시 공영장례 지원 조례.txt',
 '김제시 공영장례 지원 조례.txt',
 '김포시 공영장례 지원에 관한 조례.txt',
 '김해시 공영장례 지원에 관한 조례.txt',
 '나주시 공영장례 지원에 관한 조례.txt',
 '남양주시 공영장례 지원에 관한 조례.txt',
 '남원시 공영장례 지원 조례.txt',
 '남해군 공영장례 지원 조례.txt',
 '논산시 무연고 사망자 공영장례 지원에 관한 조례.txt',
 '단양군 공영장례 지원에 관한 조례.txt',
 '대구광역시 공영장례 지원 조례.txt',
 '대구광역시 군위군 공영장례에 관한 조례.txt',
 '대구광역시 남구 공영장례 지원에 관한

In [4]:
len(ordinances_path_list)

148